In [2]:
from euroleague_api.play_by_play_data import PlayByPlay 
from euroleague_api.game_stats import GameStats
import warnings
import pandas as pd
import statsmodels.api as sm
import numpy as np
# Masquer tous les warnings
warnings.filterwarnings("ignore")
import random

C:\Users\guill\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [64]:
def transform_elo(elo_classique):
    """
    Transforme le classement Elo classique en un classement sur l'échelle de 50 à 100.
    
    :param elo_classique: Classement Elo classique
    :return: Classement sur l'échelle de 50 à 100
    """
    return 50 + (elo_classique - 1200) / 400 * 50

def elo_update(elo_a, elo_b, score, k,points):
    """
    Met à jour les classements Elo pour deux joueurs après un match.
    
    :param elo_a: Classement actuel du joueur A (sur l'échelle 50-100)
    :param elo_b: Classement actuel du joueur B (sur l'échelle 50-100)
    :param score: Score du joueur A (1 pour victoire, 0.5 pour match nul, 0 pour défaite)
    :param k: Coefficient K pour le système de classement
    :return: (nouveau elo_a, nouveau elo_b)
    """
    # Convertir les classements sur l'échelle de 50 à 100 en classements classiques
    elo_a_classique = (elo_a - 50) * 400 / 50 + 1200
    elo_b_classique = (elo_b - 50) * 400 / 50 + 1200
    
    # Calculer le score attendu
    expected_a = 1 / (1 + 10 ** ((elo_b_classique - elo_a_classique) / 400))
    expected_b = 1 / (1 + 10 ** ((elo_a_classique - elo_b_classique) / 400))
    
    # Mettre à jour les classements
    nouveau_elo_a_classique = elo_a_classique + k * points* (score - expected_a)
    nouveau_elo_b_classique = elo_b_classique + k * points*((1 - score) - expected_b)
    
    # Convertir les classements mis à jour en échelle de 50 à 100
    nouveau_elo_a = transform_elo(nouveau_elo_a_classique)
    nouveau_elo_b = transform_elo(nouveau_elo_b_classique)
    
    return nouveau_elo_a/elo_a,nouveau_elo_b/elo_b
 

In [66]:

# Exemple d'utilisation
elo_a = 90  
elo_b = 75   
score = 0
points = 2
k = 0.2

nouveau_elo_a, nouveau_elo_b = elo_update(elo_a, elo_b, score, k,points)
print(f"Nouveau Elo A: {nouveau_elo_a:.5f}, Nouveau Elo B: {nouveau_elo_b:.5f}")


Nouveau Elo A: 0.99963, Nouveau Elo B: 1.00044


In [82]:
competition ="euroleague"
season = 2024

five = pd.DataFrame(columns=[])
points = pd.DataFrame(columns=[])
elo_data = pd.DataFrame(columns=[])
if competition == "euroleague" :
    pbp_ = PlayByPlay(competition="E")
    gs_ = GameStats(competition="E")

else :
    pbp_ = PlayByPlay(competition="U")
    gs_ = GameStats(competition="U")


for gc in range(306) : 
    gamecode=gc + 1

    try :
            
        gs = gs_.get_game_report(season=season, game_code=gamecode)
        pbp = pbp_.get_game_play_by_play_data(season=season, gamecode=gamecode)
        HOME_TEAM = gs["local.club.code"].tolist()[0]
        AWAY_TEAM = gs["road.club.code"].tolist()[0]

        pbp.loc[:, "NUMBEROFPLAY"] = range(len(pbp))

        sub = pbp[
            (pbp['PLAYINFO'] == 'Out') |
            (pbp['PLAYINFO'] == 'In')
        ].reset_index(drop = True)


        unique_players = sub[['PLAYER_ID', 'CODETEAM']].drop_duplicates()
        # Filtrer les événements "IN" et "OUT"
        in_events = sub[sub['PLAYTYPE'] == 'IN']
        out_events = sub[sub['PLAYTYPE'] == 'OUT']

        # Trouver le premier IN pour chaque joueur/équipe
        first_in = in_events.groupby(['PLAYER_ID', 'CODETEAM']).first().reset_index()
        first_in = first_in[['PLAYER_ID', 'CODETEAM', 'NUMBEROFPLAY']].rename(columns={'NUMBEROFPLAY': 'FIRST_IN'})

        # Trouver le premier OUT pour chaque joueur/équipe
        first_out = out_events.groupby(['PLAYER_ID', 'CODETEAM']).first().reset_index()
        first_out = first_out[['PLAYER_ID', 'CODETEAM', 'NUMBEROFPLAY']].rename(columns={'NUMBEROFPLAY': 'FIRST_OUT'})

        # Fusionner les premiers événements IN et OUT avec tous les joueurs
        result = pd.merge(unique_players, first_in, on=['PLAYER_ID', 'CODETEAM'], how='left')
        result = pd.merge(result, first_out, on=['PLAYER_ID', 'CODETEAM'], how='left')


        # Filtrer le DataFrame
        df_pbp = sub[
            (sub['PLAYINFO'] == 'Out') |
            (sub['PLAYINFO'] == 'In')
        ].reset_index(drop = True)

        df_pbp = df_pbp[["CODETEAM","PLAYER_ID","NUMBEROFPLAY","PLAYTYPE"]]



        result["STARTER"] = (result['FIRST_IN'] > result['FIRST_OUT']) | (result['FIRST_IN'].isna() & result['FIRST_OUT'].isna())| (result['FIRST_IN'].isna())
        starter = result[result["STARTER"]].reset_index(drop=True)
        starter = starter[starter['PLAYER_ID'] != ""]

        df_pbp_start = pd.DataFrame({
            "NUMBEROFPLAY": [0] * len(starter),
            "CODETEAM": starter["CODETEAM"].to_list(),
            "PLAYER_ID": starter["PLAYER_ID"].to_list(),
            "PLAYTYPE": ["IN"] * len(starter),
        })

        sub = sub[df_pbp_start.columns.tolist()]
        sub = pd.concat([df_pbp_start, sub], ignore_index=True)

        sub["P_T"] = sub["CODETEAM"] + "_" + sub["PLAYER_ID"]

        actual_ten = set()  # Utiliser un set pour une gestion plus rapide
        historique_ten = []

        for index, row in sub.iterrows():
            player_id = row["P_T"]
            if row["PLAYTYPE"] == "IN":
                actual_ten.add(player_id)  # Ajouter le joueur
            elif row["PLAYTYPE"] == "OUT":
                actual_ten.discard(player_id)  # Retirer le joueur si présent

            historique_ten.append(sorted(actual_ten))  # Append une liste triée des joueurs

        sub["TEN"] = historique_ten

        sub = sub[sub["TEN"].apply(lambda x: len(x) == 10)]

        sub['FIVE_HOME'] = sub['TEN'].apply(lambda x: sorted([elem[4:] for elem in x if elem.startswith(f"{HOME_TEAM}_")]))
        sub['FIVE_AWAY'] = sub['TEN'].apply(lambda x: sorted([elem[4:] for elem in x if elem.startswith(f"{AWAY_TEAM}_")]))

        sub["Season"] = season

        sub["Gamecode"] = gamecode

        sub.rename(columns={'NUMBEROFPLAY': 'START'}, inplace=True)


        sub['END'] = (sub['START'].shift(-1).fillna(max(pbp["NUMBEROFPLAY"])) -1).astype(int)


        sub = sub[["Season","Gamecode","START","END","FIVE_HOME","FIVE_AWAY"]]
        #five = pd.concat([five,sub])
        sub2 = pbp[
            (pbp['PLAYTYPE'] == '2FGM') |
            (pbp['PLAYTYPE'] == '3FGM')|
            (pbp['PLAYTYPE'] == 'FTM')
        ].reset_index(drop = True)

        sub2['POINTS'] = sub2['PLAYTYPE'].replace({'2FGM': 2, '3FGM': 3, 'FTM': 1})
        sub2 = sub2[["Season","Gamecode","NUMBEROFPLAY","CODETEAM","POINTS"]]
        #points = pd.concat([points,sub2])

        result = sub2.merge(sub, on=['Season', 'Gamecode'], how='left')
        result = result[(result['NUMBEROFPLAY'] > result['START']) & (result['NUMBEROFPLAY'] < result['END'])].reset_index(drop=True)
        result['CODETEAM2'] = result['CODETEAM'].apply(lambda x: 'HOME' if x == HOME_TEAM else 'AWAY')

        elo_data = pd.concat([elo_data,result]).reset_index(drop=True)
    except :
        pass

In [83]:
joueurs = list(set(element for sublist in elo_data['FIVE_HOME'].tolist() + elo_data['FIVE_AWAY'].tolist() for element in sublist))

dictionnaire_joueurs = {joueur: [75] for joueur in joueurs}

for i in range(10) : 
    for index, row in elo_data.iterrows():
        elo_team_home = np.mean([dictionnaire_joueurs[joueur][-1] for joueur in row["FIVE_HOME"]])
        elo_team_away = np.mean([dictionnaire_joueurs[joueur][-1] for joueur in row["FIVE_AWAY"]])
        points = row["POINTS"]
        score = int(row["CODETEAM2"] == "HOME")
        coeff_home,coeff_away = elo_update(elo_team_home, elo_team_away, score, 0.2,points)
        # Mise à jour des valeurs des joueurs pour l'équipe HOME
        for joueur in row["FIVE_HOME"]:
            nouvelle_valeur = dictionnaire_joueurs[joueur][-1] * coeff_home
            dictionnaire_joueurs[joueur].append(nouvelle_valeur)

        # Mise à jour des valeurs des joueurs pour l'équipe AWAY
        for joueur in row["FIVE_AWAY"]:
            nouvelle_valeur = dictionnaire_joueurs[joueur][-1] * coeff_away
            dictionnaire_joueurs[joueur].append(nouvelle_valeur)


# Création du DataFrame à partir du dictionnaire des joueurs
df_joueurs = pd.DataFrame({
    'Joueur': list(dictionnaire_joueurs.keys()),
    'Derniere_Valeur': [valeurs[-1] for valeurs in dictionnaire_joueurs.values()]
})
df_joueurs = df_joueurs.sort_values(by="Derniere_Valeur",ascending=False)
df_joueurs

,Joueur,Derniere_Valeur
127,P004863,86.793869
192,PJDR,85.800311
80,P005928,85.624185
138,P011218,84.368646
207,P000925,84.240163
...,...,...
122,P005356,66.816735
106,P012730,66.598390
173,P007550,66.425371
116,P008850,66.398785
